<a href="https://colab.research.google.com/github/ncorpron/Restaurant_Inventory/blob/main/Restaurant_Inventory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
!pip install streamlit pandas plotly openpyxl pyngrok --upgrade -q


In [41]:
%%writefile utils.py
import pandas as pd
import numpy as np

def load_data():
    data = {
        "Item": ["Chicken Breast", "Beef", "Rice", "Milk", "Lettuce", "Eggs", "Cheese", "Tomatoes", "Fish", "Yogurt"],
        "Category": ["Meat", "Meat", "Grain", "Dairy", "Vegetable", "Dairy", "Dairy", "Vegetable", "Meat", "Dairy"],
        "Location": ["Downtown", "Downtown", "Uptown", "Uptown", "Downtown", "Downtown", "Uptown", "Uptown", "Downtown", "Uptown"],
        "Beginning Inventory": [50, 40, 100, 30, 20, 5, 15, 25, 10, 20],
        "Purchases": [20, 10, 50, 15, 10, 2, 5, 10, 5, 10],
        "Usage": [40, 30, 70, 20, 15, 5, 10, 20, 7, 15],
        "Price_per_Unit": [3.5, 5.0, 1.0, 0.8, 0.5, 0.2, 2.0, 1.2, 4.0, 1.0],
        "Supplier": ["Supplier A", "Supplier B", "Supplier A", "Supplier C", "Supplier B",
                     "Supplier C", "Supplier B", "Supplier A", "Supplier C", "Supplier B"],
        "Lead_Time_Days": [3, 5, 2, 2, 4, 3, 5, 2, 3, 2]
    }
    inventory = pd.DataFrame(data)

    # Add historical usage
    months = ["June", "July", "August"]
    usage_history = {"Item": inventory["Item"].tolist()}
    for month in months:
        usage_history[month] = [np.random.randint(20, 50) for _ in range(len(inventory))]
    usage_df = pd.DataFrame(usage_history)

    inventory = inventory.merge(usage_df, on="Item")
    inventory["Avg_Usage"] = inventory[months].mean(axis=1)

    return inventory, usage_df, months


def recalc(df, months=None):
    df["End Inventory"] = df["Beginning Inventory"] + df["Purchases"] - df["Usage"]
    df["Stock Value"] = df["End Inventory"] * df["Price_per_Unit"]
    df["Reorder Amount"] = df["End Inventory"].apply(lambda x: max(20 - x, 0))
    if months:
        df["Days_Left"] = df["End Inventory"] / (df["Avg_Usage"] / 30)
    else:
        df["Days_Left"] = df["End Inventory"]
    df["Predicted Reorder"] = df["End Inventory"].apply(lambda x: max(0, 20 - x))
    df["Suggested_Order"] = ((df["Avg_Usage"]/30) * df["Lead_Time_Days"] + df["Predicted Reorder"]).round()
    return df


Overwriting utils.py


In [42]:
%%writefile Inventory.py
import streamlit as st
from utils import load_data, recalc
import pandas as pd

def inventory_page():
    # -------------------- Banner with Title --------------------
    banner_url = "https://images.unsplash.com/photo-1600891964599-f61ba0e24092?auto=format&fit=crop&w=1350&q=80"
    st.markdown(
        f"""
        <div style="
            position: relative;
            text-align: center;
            color: white;
        ">
            <img src="{banner_url}" style="width:100%; height:200px; object-fit:cover; border-radius:8px;">
            <div style="
                position: absolute;
                top: 50%;
                left: 50%;
                transform: translate(-50%, -50%);
                background-color: rgba(0,0,0,0.5);
                padding: 10px 20px;
                border-radius: 5px;
                font-size: 32px;
                font-weight: bold;
                font-family: sans-serif;
            ">
                Restaurant Inventory Overview
            </div>
        </div>
        """,
        unsafe_allow_html=True
    )

    # -------------------- Load Data --------------------
    inventory, usage_df, months = load_data()
    filtered_inventory = recalc(inventory, months)

    # -------------------- Key Metrics --------------------
    st.subheader("📊 Key Metrics")
    total_stock_value = filtered_inventory["Stock Value"].sum()
    avg_end_inventory = filtered_inventory["End Inventory"].mean()
    most_used_item = filtered_inventory.loc[filtered_inventory["Usage"].idxmax(), "Item"]
    highest_value_item = filtered_inventory.loc[filtered_inventory["Stock Value"].idxmax(), "Item"]

    col1, col2, col3, col4 = st.columns(4)
    col1.metric("Total Stock Value ($)", round(total_stock_value, 2))
    col2.metric("Average End Inventory", round(avg_end_inventory, 1))
    col3.metric("Most Used Item", most_used_item)
    col4.metric("Highest Value Item", highest_value_item)

    # -------------------- Full Inventory Table --------------------
    st.subheader("Full Inventory Table")
    st.dataframe(filtered_inventory)

    # -------------------- End Inventory by Item (Low / Medium / High) --------------------
    st.subheader("End Inventory by Item (Low / Medium / High)")
    low_stock_threshold = st.sidebar.slider("Low Stock Threshold", 1, 50, 10)

    def stock_status(val):
        if val < low_stock_threshold:
            return "🟥 Low"
        elif val < low_stock_threshold * 2:
            return "🟨 Medium"
        else:
            return "🟩 High"

    filtered_inventory["Stock_Status"] = filtered_inventory["End Inventory"].apply(stock_status)

    # Toggle filter
    selected_status = st.radio(
        "🔍 Highlight a stock status:",
        ["All", "🟥 Low", "🟨 Medium", "🟩 High"],
        horizontal=True
    )

    if selected_status != "All":
        display_df = filtered_inventory[filtered_inventory["Stock_Status"] == selected_status]
    else:
        display_df = filtered_inventory.copy()

    # Mini metrics for filtered view
    col1, col2, col3 = st.columns(3)
    col1.metric("Total Items", len(display_df))
    col2.metric("Avg End Inventory", round(display_df["End Inventory"].mean(), 1))
    col3.metric("Most Critical Item", display_df.loc[display_df['End Inventory'].idxmin(), 'Item'])

    # Show table and allow sorting
    st.dataframe(display_df[["Item","Category","Location","End Inventory","Stock_Status","Suggested_Order"]]
                 .sort_values("End Inventory", ascending=True))

    # Download CSV
    csv = display_df.to_csv(index=False).encode("utf-8")
    st.download_button("⬇️ Download Stock Status CSV", data=csv, file_name="stock_status.csv", mime="text/csv")



Overwriting Inventory.py


In [43]:
%%writefile Charts.py
import streamlit as st
import plotly.express as px
import pandas as pd
from utils import load_data, recalc

def charts_page():
    st.title("📊 Charts & Inventory Insights")

    # -------------------- Banner Image --------------------
    st.markdown(
        """
        <div style="
            background-image: url('https://images.unsplash.com/photo-1565299624946-b28f40a0ae38');
            background-size: cover;
            padding: 50px;
            border-radius: 10px;
            text-align: center;
            color: white;
            font-size: 36px;
            font-weight: bold;">
            🍷 Food & wine Inventory Dashboard
        </div>
        """,
        unsafe_allow_html=True
    )

    # -------------------- Load Data --------------------
    inventory, usage_df, months = load_data()
    filtered_inventory = recalc(inventory, months)

    # -------------------- 0. Low/Medium/High Stock Status --------------------
    st.subheader("0. Stock Status Overview (Low / Medium / High)")

    low_stock_threshold = st.sidebar.slider("Low Stock Threshold", 1, 50, 10)

    def stock_status(val):
        if val < low_stock_threshold:
            return "🟥 Low"
        elif val < low_stock_threshold * 2:
            return "🟨 Medium"
        else:
            return "🟩 High"

    filtered_inventory["Stock_Status"] = filtered_inventory["End Inventory"].apply(stock_status)

    # Toggle filter
    selected_status = st.radio(
        "🔍 Highlight a stock status:",
        ["All", "🟥 Low", "🟨 Medium", "🟩 High"],
        horizontal=True
    )

    if selected_status != "All":
        display_df = filtered_inventory[filtered_inventory["Stock_Status"] == selected_status]
    else:
        display_df = filtered_inventory.copy()

    # -------------------- Mini Metrics --------------------
    col1, col2, col3 = st.columns(3)
    col1.metric("Total Items", len(display_df))
    col2.metric("Avg End Inventory", round(display_df["End Inventory"].mean(), 1))
    col3.metric("Most Critical Item", display_df.loc[display_df['End Inventory'].idxmin(), 'Item'])

    # -------------------- Stock Table --------------------
    st.dataframe(display_df[["Item","Category","Location","End Inventory","Stock_Status","Suggested_Order"]]
                 .sort_values("End Inventory", ascending=True))

    # Download CSV
    csv = display_df.to_csv(index=False).encode("utf-8")
    st.download_button("⬇️ Download Stock Status CSV", data=csv, file_name="stock_status.csv", mime="text/csv")

    # -------------------- 1. Usage vs End Inventory --------------------
    st.subheader("1. Usage vs End Inventory")
    fig1 = px.bar(filtered_inventory, x="Item", y=["Usage", "End Inventory"], barmode="group",
                  color_discrete_sequence=px.colors.qualitative.Pastel)
    st.plotly_chart(fig1, use_container_width=True)

    # -------------------- 2. Stock Value Distribution --------------------
    st.subheader("2. Stock Value Distribution")
    fig2 = px.pie(filtered_inventory, names="Item", values="Stock Value", hole=0.3,
                  color_discrete_sequence=px.colors.sequential.Teal)
    st.plotly_chart(fig2, use_container_width=True)

    # -------------------- 3. Price vs Stock Value Scatter --------------------
    st.subheader("3. Price vs Stock Value Scatter")
    fig3 = px.scatter(filtered_inventory, x="Price_per_Unit", y="Stock Value", size="End Inventory",
                      color="Category", color_discrete_sequence=px.colors.qualitative.Bold)
    st.plotly_chart(fig3, use_container_width=True)

    # -------------------- 4. Avg Usage Line Chart --------------------
    st.subheader("4. Avg Usage Line Chart")
    fig4 = px.line(filtered_inventory, x="Item", y="Avg_Usage", markers=True, color_discrete_sequence=px.colors.sequential.Plasma)
    st.plotly_chart(fig4, use_container_width=True)


Overwriting Charts.py


In [44]:
%%writefile Reports.py
from utils import load_data, recalc
import streamlit as st
import io

def reports_page():
    # -------------------- Banner --------------------
    banner_url = "https://images.unsplash.com/photo-1565299624946-b28f40a0ae38?auto=format&fit=crop&w=1500&q=80"  # New image
    st.markdown(
        f"""
        <div style="position: relative; text-align: center; color: white;">
            <img src="{banner_url}" style="width:100%; height:200px; object-fit:cover; border-radius:10px;">
            <h2 style="position: absolute; top: 50%; left: 50%; transform: translate(-50%, -50%);
                       font-family: 'Arial', sans-serif; font-weight:700; font-size:38px;
                       background-color: rgba(0, 0, 0, 0.45); padding: 10px 25px; border-radius: 5px;">
                Restaurant Inventory Reports
            </h2>
        </div>
        """,
        unsafe_allow_html=True
    )

    # -------------------- Load Data --------------------
    inventory, usage_df, months = load_data()
    filtered_inventory = recalc(inventory, months)

    # -------------------- Export CSV --------------------
    st.subheader("Export Inventory Data")
    csv = filtered_inventory.to_csv(index=False).encode('utf-8')
    st.download_button(
        label="⬇️ Download CSV",
        data=csv,
        file_name='inventory.csv',
        mime='text/csv'
    )

    # -------------------- Export Excel --------------------
    output = io.BytesIO()
    filtered_inventory.to_excel(output, index=False, engine='openpyxl')
    st.download_button(
        label="⬇️ Download Excel",
        data=output.getvalue(),
        file_name='inventory.xlsx',
        mime='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
    )

    # -------------------- Revenue & Profit --------------------
    markup_percent = st.sidebar.number_input(
        "Revenue Markup (%)",
        min_value=0,
        max_value=200,
        value=50,
        step=5
    )
    filtered_inventory["Revenue_Potential_Marked"] = filtered_inventory["End Inventory"] * filtered_inventory["Price_per_Unit"] * (1 + markup_percent/100)
    filtered_inventory["Potential_Profit_Marked"] = filtered_inventory["Revenue_Potential_Marked"] - filtered_inventory["Stock Value"]

    st.subheader("Revenue & Profit Projection")
    st.dataframe(filtered_inventory[["Item","End Inventory","Revenue_Potential_Marked","Potential_Profit_Marked"]])


Overwriting Reports.py


In [45]:
%%writefile Alerts.py
from utils import load_data, recalc
import streamlit as st

def alerts_page():
    # -------------------- Banner --------------------
    banner_url = "https://images.unsplash.com/photo-1586190848861-99aa4a171e90?auto=format&fit=crop&w=1500&q=80"  # New image for alerts
    st.markdown(
        f"""
        <div style="position: relative; text-align: center; color: white;">
            <img src="{banner_url}" style="width:100%; height:200px; object-fit:cover; border-radius:10px;">
            <h2 style="position: absolute; top: 50%; left: 50%; transform: translate(-50%, -50%);
                       font-family: 'Arial', sans-serif; font-weight:700; font-size:38px;
                       background-color: rgba(0, 0, 0, 0.45); padding: 10px 25px; border-radius: 5px;">
                Inventory Alerts & Notifications
            </h2>
        </div>
        """,
        unsafe_allow_html=True
    )

    # -------------------- Load Data --------------------
    inventory, usage_df, months = load_data()
    filtered_inventory = recalc(inventory, months)

    st.subheader("Low Stock Alerts")
    low_stock_threshold = st.sidebar.slider("Low Stock Threshold", 1, 50, 10)
    low_stock = filtered_inventory[filtered_inventory["End Inventory"] < low_stock_threshold]

    if not low_stock.empty:
        st.warning("Low Stock Items:")
        st.dataframe(low_stock[["Item","End Inventory","Reorder Amount","Category","Location"]])
        if st.button("Send Low Stock Alert (Simulated)"):
            st.success("Alerts sent via Email/SMS simulation!")

    st.subheader("Predictive Alerts")
    alert_days = st.sidebar.slider("Predictive Alert Threshold (days left)", 1, 60, 15)
    predicted_alerts = filtered_inventory[filtered_inventory["Days_Left"] < alert_days]

    if not predicted_alerts.empty:
        st.error("Items Predicted to Run Out Soon:")
        st.dataframe(predicted_alerts[["Item","End Inventory","Days_Left","Predicted Reorder"]])
        if st.button("Send Predictive Alert (Simulated)"):
            st.success("Predictive alerts sent via Email/SMS simulation!")


Overwriting Alerts.py


In [69]:
%%writefile main_app.py
import streamlit as st
from Inventory import inventory_page
from Charts import charts_page
from Reports import reports_page
from Alerts import alerts_page

st.set_page_config(page_title="Restaurant Inventory Dashboard", layout="wide")

# -----------------------------
# Session state defaults
# -----------------------------
if "page" not in st.session_state:
    st.session_state.page = "Inventory"

# Default values for thresholds
defaults = {
    "low_threshold": 10,
    "high_threshold": 50
}

for key, val in defaults.items():
    if key not in st.session_state:
        st.session_state[key] = val

# -----------------------------
# Sidebar CSS: wood background + button styling
# -----------------------------
st.markdown("""
<style>
[data-testid="stSidebar"] {
    background-image: url('https://wallpapercave.com/wp/KnY66yi.jpg');
    background-size: cover;
    background-repeat: no-repeat;
    background-position: center;
    height: 100vh;
}
div.stButton > button {
    color: black;
    font-weight: bold;
    background-color: white;
    border-radius: 8px;
    width: 100%;
    margin-bottom: 5px;
}
</style>
""", unsafe_allow_html=True)

# -----------------------------
# Sidebar: Navigation
# -----------------------------
st.sidebar.markdown(
    '<div style="font-weight:bold; color:black; background-color:rgba(255,255,255,0.8); padding:8px 10px; border-radius:6px; text-align:center;">Navigation</div>',
    unsafe_allow_html=True
)

pages = ["Inventory", "Charts", "Reports", "Alerts"]
for p in pages:
    if st.sidebar.button(p, key=f"page_{p}"):
        st.session_state.page = p

# -----------------------------
# Sidebar: Thresholds
# -----------------------------
st.sidebar.markdown(
    '<div style="font-weight:bold; color:black; background-color:rgba(255,255,255,0.8); padding:8px 10px; border-radius:6px; text-align:center;">Thresholds</div>',
    unsafe_allow_html=True
)

# Only 2 thresholds
thresholds = [
    ("Low Threshold", "low_threshold", 0, 100),
    ("High Threshold", "high_threshold", 0, 200)
]

for label, key, min_val, max_val in thresholds:
    st.sidebar.markdown(
        f'<div style="color:black; font-weight:bold; background-color:white; padding:3px 6px; border-radius:4px; margin-top:5px; margin-bottom:3px;">{label}</div>',
        unsafe_allow_html=True
    )
    st.session_state[key] = st.sidebar.slider(
        "",
        min_val,
        max_val,
        st.session_state[key],
        key=f"slider_{key}"
    )

# -----------------------------
# Page routing
# -----------------------------
if st.session_state.page == "Inventory":
    inventory_page()
elif st.session_state.page == "Charts":
    charts_page()
elif st.session_state.page == "Reports":
    reports_page()
elif st.session_state.page == "Alerts":
    alerts_page()


Overwriting main_app.py


In [70]:
# 1️⃣ Install pyngrok if needed
!pip install pyngrok --quiet
!pip install streamlit --quiet

# 2️⃣ Set your ngrok authtoken
!ngrok authtoken "31xcLxKwT5osADQdP1r6JEIpbu9_5iiDf8vxVaypwrqDkYpk9"

# 3️⃣ Import modules
from pyngrok import ngrok, conf
import time
import subprocess
import signal
import psutil

# 4️⃣ Kill any existing ngrok tunnels
try:
    ngrok.kill()
except:
    pass  # ignore if no tunnels exist

# 5️⃣ Kill existing Streamlit processes on port 8501
for proc in psutil.process_iter():
    try:
        if "streamlit" in proc.name().lower():
            proc.kill()
    except:
        pass

# 6️⃣ Start Streamlit as a background process
process = subprocess.Popen(
    ["streamlit", "run", "main_app.py", "--server.port=8501", "--server.headless=true"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

# 7️⃣ Wait for Streamlit to start
time.sleep(5)

# 8️⃣ Start ngrok tunnel to Streamlit
public_url = ngrok.connect(8501)
print("Your Streamlit app is live at:", public_url)


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Your Streamlit app is live at: NgrokTunnel: "https://d5cac2325888.ngrok-free.app" -> "http://localhost:8501"
